In [1]:
import pandas as pd
import json
import os
import glob

In [2]:
df = pd.read_csv('C:/Users/eovit/Python/Kaggle/2020-05-28/metadata.csv', low_memory= False)

In [3]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [4]:
df.shape

(136067, 19)

In [5]:
with open('C:/Users/eovit/Python/Kaggle/2020-05-28/document_parses/pdf_json/000a0fc8bbef80410199e690191dc3076a290117.json', 'rb') as f:
    dat  = json.load(f)

In [28]:
def json_conv(json_obj):
    """
    This function handles the native JSON schema from the COVID pdf-conversions\
    
    Input - > Json object
    
    Output - > df of title, author, abstract, full text
        i.e it is a list of length 4
        
    This function does not catch all exceptions, like colons, admission of the keyword being looked at

        
    """
    paper_id = ''
    information = []
    title = ''
    all_auths = []
    all_body = []
    abst = ''
    if json_obj['paper_id']:
        paper_id += json_obj['paper_id']

    if json_obj['metadata']:
            
        if json_obj['metadata']['title']:
            title += str(json_obj['metadata']['title'])
            
        #handling naming conventions in the json 
        for i in json_obj['metadata']['authors']:
            prop_name = ''
            if i['middle']:
                prop_name += str(i['first']) + ' ' +  str(i['middle']) + ' ' + str(i['last'])
            else:
                prop_name += str(i['first']) +  ' ' + str(i['last'])
            all_auths.append(prop_name)
        
    #uncomment this for the PDF_jsons, comment out for the pmc_jsons
    #lets drop this so we can get ALL THE files.
        if json_obj['abstract'] == True:
            for j in json_obj['abstract']:
                if j['text']:
                    abst += str(j['text'])

                    
                    
        if json_obj['body_text']:
            body_txt = ''
            for l in json_obj['body_text']:
                if l['text']:
                    body_txt += str(l['text'])
            all_body.append(body_txt)
            
            
    information.append(title)
    information.append(all_auths)
    information.append(abst)
    information.append(body_txt)
    return information

In [29]:
def json_opener(path):
    #wildcard to open all jsons
    all_data = []
    #abstract to be added for PDF, removed for pmc
    col_names = ['title', 'authors', 'abstract', 'all_body']

    for filename in glob.glob(os.path.join(path, '*.json')):
        with open(filename, 'rb') as f:
            data = json.load(f)
            d_con = json_conv(data)
        all_data.append(d_con)
    df = pd.DataFrame(all_data, columns = col_names)
    return df

In [30]:
if __name__ == '__main__':
    
    #opening all of the files both the PDF and the PMC files/ PMC files do not have abstracts so be careful
    path = 'C:/Users/eovit/Python/Kaggle/2020-05-28/document_parses/pdf_json/'

    json_extract = json_opener(path)

    df = pd.DataFrame(json_extract)
    df.to_csv('C:/Users/eovit/Python/Kaggle/research_pdfpapers_v2.csv')